In [1]:
import pandas as pd
import numpy as np

tl=pd.read_csv('../Rscripts/data/90360AAAE_PR35_M65705_11_005.03.csv',skiprows=[1,2,3], low_memory=False, error_bad_lines=False)

In [2]:
tl.head()

,dlog_version,setup time,mode code,lot id,part type,sublot id,facility id,start time,node name,datafile path,...,MatSlopeNormal50,MatSlopeClampHigh50,Ram_With_Rom_Program_Iddq_Mode,Ram_With_Ram_Program_Iddq_Mode,RamRom_Program_InIddqMode_vVdecSet,RamRam_Program_InIddqMode_vVdecSet,HVAS,Hvas_Vdec_Voltage,TempSensorSum_fw,Unnamed: 567
0,DAT_LINUX,2018-02-06T16:30:20.970Z,PROD,M65705,NaN,99,NaN,NaN,Nena,/mnt/linuxtesters/testlog/90360AAAE.M65705/PR3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.0,NaN
1,DAT_LINUX,2018-02-06T16:30:20.970Z,PROD,M65705,NaN,99,NaN,NaN,Nena,/mnt/linuxtesters/testlog/90360AAAE.M65705/PR3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.0,NaN
2,DAT_LINUX,2018-02-06T16:30:20.970Z,PROD,M65705,NaN,99,NaN,NaN,Nena,/mnt/linuxtesters/testlog/90360AAAE.M65705/PR3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.0,NaN
3,DAT_LINUX,2018-02-06T16:30:20.970Z,PROD,M65705,NaN,99,NaN,NaN,Nena,/mnt/linuxtesters/testlog/90360AAAE.M65705/PR3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,NaN
4,DAT_LINUX,2018-02-06T16:30:20.970Z,PROD,M65705,NaN,99,NaN,NaN,Nena,/mnt/linuxtesters/testlog/90360AAAE.M65705/PR3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,NaN


Check how many duplicates there are in the data frame. To run the GDBC (Good Die Bad Cluster) algorithm we want to make sure we have 1 unique x, y, bincode combination.

In [3]:
gt1=tl[['x coord','y coord',"BIN"]].groupby(['x coord','y coord']).size()>1
sum(gt1)

139

Now group by x and y coordinates and select only the first one to create the unique x, y, bincode combination.

In [4]:
tlu=tl[['x coord','y coord',"BIN"]].groupby(['x coord','y coord']).head(1)

Check that we have a unique combination.

In [5]:
gt1=tlu[['x coord','y coord',"BIN"]].groupby(['x coord','y coord']).size()>1
sum(gt1)

0

In [9]:
def countnnfails(xnn,ynn,df):
    dfc1=df['x coord']>=xnn-1
    dfc2=df['x coord']<=xnn+1
    dfc3=df['y coord']>=ynn-1
    dfc4=df['y coord']<=ynn+1
    dfc5=np.logical_or(df['x coord']!=xnn,df['y coord']!=ynn)
    
    dfsub=df[['BIN']][dfc1 & dfc2 & dfc3 & dfc4 & dfc5]
    
    return(sum(np.logical_and(dfsub['BIN']>349,dfsub['BIN']<500)))

In [10]:
gdbc=tlu.apply(lambda x: countnnfails(x['x coord'],x['y coord'],tlu),axis=1)

In [11]:
gdbc.value_counts()

0    7314
1     472
2      90
3      21
4       6
5       1
dtype: int64

In [12]:
crs=pd.DataFrame({'x coord' : pd.Series([40, 60, 79, 95, 40]),
                   'y coord' : pd.Series([17, 15, 67, 21, 12]),
                   'cr'      : pd.Series([1,1,1,1,1])})
                                         

In [13]:
crs

,cr,x coord,y coord
0,1,40,17
1,1,60,15
2,1,79,67
3,1,95,21
4,1,40,12


In [14]:
tlu['gdbc']=gdbc

dfm=pd.merge(tlu,crs,how="left")

In [15]:
dfm=dfm.fillna(0)

In [16]:
dfm['cr'].value_counts()

0.0    7899
1.0       5
Name: cr, dtype: int64

In [17]:
dfm[['x coord','y coord','cr','gdbc']][dfm['cr']==1]

,x coord,y coord,cr,gdbc
513,60,15,1.0,1
587,40,12,1.0,0
699,40,17,1.0,1
1080,95,21,1.0,2
6353,79,67,1.0,2


In [18]:
dfm.groupby(['cr','gdbc']).size()

cr   gdbc
0.0  0       7313
     1        470
     2         88
     3         21
     4          6
     5          1
1.0  0          1
     1          2
     2          2
dtype: int64

In [19]:
gdbc[gdbc>0].count()

590